## Schur complement via block LU decomposition

In [ ]:
"""
Naive LU decomposition (no pivoting) from
[Bindel's CS6210 class notes](http://www.cs.cornell.edu/~bindel/class/cs6210-f12/notes/lec09.pdf).

Beware A[1,1] = 0...
"""
function mylu(A)
    m,n = size(A)
    for j = 1:n-1
        A[j+1:n,j] = A[j+1:n,j]/A[j,j]
        A[j+1:n,j+1:n] = A[j+1:n,j+1:n] - A[j+1:n,j]*A[j,j+1:n]
    end
    return A
end

"""
Return A[i2,i2] - A[i2,i1]*inv(A[i1,i1])*A[i1,i2].

Uses block LU decomposition.
"""
function schurcomp(A,i1,i2)
    A11 = lufact(A[i1,i1])
    if issparse(A)
        L11,U11,p,q,Rs = A11[:(:)]
        P = pvec2mat(p)
        Q = pvec2mat(q)
        Rs = diagm(Rs)
        
        L21 = (A[i2,i1]*Q)/UpperTriangular(U11)
        U12 = LowerTriangular(L11)\(P*Rs*A[i1,i2])
        
        S = A[i2,i2] - L21*U12
        return S,(P,Q,Rs)
    else
        P = pvec2mat(A11[:p])
        println(A11[:p])
        L11 = A11[:L]
        U11 = A11[:U]

        L21 = A[i2,i1]/UpperTriangular(U11)
        U12 = LowerTriangular(L11)\(P*A[i1,i2])

        S = A[i2,i2] - L21*U12
        return S
    end
end

function pvec2mat(p)
    n = length(p)
    P = zeros(n,n)
    for i in 1:n
        P[i,:] = ej(n,p[i])
    end
    return sparse(P)
end

function ej(n,i)
    v = zeros(n)
    v[i] = 1.0
    return v
end

In [ ]:
A = rand(5,5)
i1 = [1;2;3]
i2 = [4;5]
schurcomp(A,i1,i2)

In [ ]:
A = sprandn(5,5,0.8)
i1 = [1;2;3]
i2 = [4;5]
S,rest = schurcomp(A,i1,i2)

A = full(A)
S - (A[i2,i2] - (A[i2,i1]/A[i1,i1])*A[i1,i2])

[full(rest[1]) full(rest[2])]

In [ ]:
S - (A[i2,i2] - (A[i2,i1]/A[i1,i1])*A[i1,i2])

In [ ]:
(A[i2,i2] - (A[i2,i1]/A[i1,i1])*A[i1,i2])

In [ ]:
A = full(A)
A[i2,i2] - (A[i2,i1]/A[i1,i1])*A[i1,i2]

In [ ]:
L11,U11,p,q,Rs = lufact(A[i1,i1])[:(:)]

P = pvec2mat(p)
Q = pvec2mat(q)

L21 = full(A[i2,i1]*Q)/UpperTriangular(U11)

In [ ]:
U12 = full(L11)\(P*(Rs.*A[i1,i2]))

S = A[i2,i2] - L21*U12

In [ ]:
Rs = [2;3;4;5;6]

(1./Rs).*Rs.*A == A